In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from datetime import date, timedelta
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
import numpy as np
from spec_constants import *
from visualization_constants import *
import plotly.express as px
import plotly.graph_objects as go
import math

# Sidewalk Labs Data Exploration

In [ ]:
# Used to filter the zones shown based on a selected zone quality when the Zones Shown selector is used.
def get_value_subset(frame_array, values_np_list, zone_quality, zones_shown):
    # The list of number of objects passing through each zone used to determine the relative object count rankings
    # of the zones.
    values_list = list(map(float, values_np_list))
    # Sort the values in descending order and take the first zones_shown values.
    if zone_quality == 'Highest Values':
        return np.isin(frame_array, sorted(values_list, reverse=True)[:zones_shown])
    # Sort the values in ascending order and take the first zones_shown values.
    elif zone_quality == 'Lowest Values':
        return np.isin(frame_array, sorted(values_list, reverse=False)[:zones_shown])
    # Sort the values in descending order and take the middle zones_shown values.
    elif zone_quality == 'Middle Values':
        lower_bound = int(ZONE_NUM/2) - int(zones_shown/2)
        return np.isin(frame_array, sorted(values_list, reverse=True)[lower_bound:lower_bound + zones_shown])

In [ ]:
def get_zone_patches(scene, heatmap_metric_array, heatmap_max, max_abs_val, show_events, display_mode, frame):
    zone_patches = []
    # For each zone, add a rectangle with the appropriate opacity given the value in the zone.
    for i in range(ROWS):
        for j in range(COLUMNS):
            # If the heatmap value is 0, no rectangle needs to be created for the area
            if heatmap_metric_array[i, j] != 0:
                # The alpha values are based on the maximum value in the heatmap frame or a user set value if
                # absolute display  mode is selected.
                if display_mode == 'Relative':
                    zone_alpha = float(heatmap_metric_array[i, j]/
                                       (1.4 * heatmap_max))
                elif display_mode == 'Absolute':
                    zone_alpha = min(0.5, float(heatmap_metric_array[i, j]/
                                       (1.4 * max_abs_val)))
                patch_color = 'blue'
                if show_events and START_DATE + timedelta(frame) in EVENT_DATES:
                    patch_color = 'green'
                # The top left corner of the rectangle is found using the stored zone coordinates.
                zone_patches.append(patches.Rectangle((ZONE_COORDINATES[scene][i * COLUMNS + j][0][0],
                                          ZONE_COORDINATES[scene][i * COLUMNS + j][0][1]),
                                         ZONE_LENGTH,ZONE_HEIGHT,fill=True,
                                        alpha=zone_alpha, facecolor=patch_color))
    # Place all constructed rectangles into a PatchCollection object for faster drawing.
    return PatchCollection(zone_patches, match_original=True)

In [ ]:
# Controls the display of advanced widgets based on whether Show Advanced Options is checked.
def set_advanced_options_view(advanced_options):
    # For each advanced widget, if Show Advanced Options is checked, then set the widget to be visible and set it to 
    # be invisible otherwise.
    for advanced_widget in advanced_widgets:
        if not advanced_options:
            advanced_widget.layout.display = 'none'
        else:
            advanced_widget.layout.display = 'flex'

In [ ]:
# If manual date input is used, then override the frame value from the date slider with the value inputted from 
# the user.
def update_chosen_date(chosen_date):
    # If nothing is set, then show the heatmap at the start.
    if chosen_date == None:
        return -1
    # If a date is set, then set the frame parameter to the be number of days since the start of the heatmap.
    else:
        frame = (chosen_date - START_DATE).days
        # If the date is outside the date range that the heatmap operates in, then show the heatmap at the start.
        if frame <= -1 or frame >= FRAMES:
            return -1
        return frame

In [ ]:
# Uses the date_slider_mode checkbox to determine whether to disable the slider, play buttons, and date input box.
# Also sets the correct frame whose data is to be processed.
def update_date_parameters(frame, date_slider_mode, chosen_date):
    # If the slider is chosen, then disabled the manual date selector box and enable the play buttons. The play buttons
    # and the slider have their disabled status linked.
    if date_slider_mode:
        date_picker_widget.disabled=True
        date_animate.disabled=False
        return frame
    # If the manual selection mode is enabled, then enable the date input box and disable the play buttons and slider.
    # Update the frame based on the value in chosen.
    else:
        frame = update_chosen_date(chosen_date)
        date_animate.value = frame
        date_picker_widget.disabled=False
        date_animate.disabled=True
        return frame

In [ ]:
# For each area, show a heatmap of the mean lingering time of each zone in the area
def show_dwell_stats(scene, object_class, metric, chosen_date, show_events, date_slider_mode, advanced_options, 
                     display_mode, max_abs_val, min_objects, weekdays, zone_quality, zones_shown, frame):
    # Set the disabled status of the play buttons, slider, and date input box and set the frame parameter to be the
    # input from the user or slider.
    frame = update_date_parameters(frame, date_slider_mode, chosen_date)
    # Show or hide the advanced options based on the whether the Show Advanced Options box is checked.
    set_advanced_options_view(advanced_options)
    # If only events are shown and date slider mode is used, then the frames move through the days in the event list.
    if show_events and date_slider_mode:
        frame = (EVENT_DATES[frame % NUM_EVENTS] - START_DATE).days
    # Convert input object class name from capitalized to non-capitalized.
    object_class = OBJECT_CLASSES_CONVERTER[object_class]
    # Convert metric array to boolean array where each entry indicates if object count is at least min_pedes
    object_count_array = HEATMAP_ARRAYS[object_class][metric][frame][SCENE_TO_NUM[scene]]
    # Filter the metric array by including only pixels corresponding to areas of pedestrian count at least min_objects
    # and set 0 to all other pixels.
    heatmap_metric_array = (object_count_array >= int(min_objects)) * object_count_array
    # Boolean array that is true for all pixels whose mean lingering value is within the zone_number mean lingering values
    # in the ROWS * COLUMNS zones found using zone_quality and it false for all other pixels.
    heatmap_values = list(object_count_array.flatten())
    # If only a certain number of zones is needed, then determine subset based on zone_quality and zones_shown.
    if zones_shown != ZONE_NUM:
        quality_boolean_array = get_value_subset(HEATMAP_ARRAYS[object_class][metric][frame][SCENE_TO_NUM[scene]], 
                                                 heatmap_values, zone_quality, zones_shown)
        # Filter the modified metric array by including only zones whose values is in the top zones_shown zones.
        heatmap_metric_array = heatmap_metric_array * quality_boolean_array
    # The maximum zone value in the heatmap used to compare all other zone values to.
    heatmap_max = np.amax(heatmap_metric_array)
    # Create a Figure object to set the area image size and an Axes object to add zone rectangles on the area image.
    fig, ax = plt.subplots(1)
    fig.set_size_inches(6, 6)
    # Overlay the heatmap on the image
    ax.imshow(PROCESSED_SCENE_IMAGES[SCENE_TO_NUM[scene]], cmap='gray', alpha=1, origin='upper')
    # If the maximum value in the heatmap is 0 or the frame is one day before the sensors start recording, all
    # zone values are 0 so this frame does not need to be processed.
    if heatmap_max != 0 and frame != -1:
        ax.add_collection(get_zone_patches(scene, heatmap_metric_array, heatmap_max, max_abs_val, 
                                           show_events, display_mode, frame))
    plt.axis('off')
    # The title shows the time range of the data using TIME_INTERVAL and frame parameters 
    start_date = START_DATE + timedelta(TIME_INTERVAL * frame)
    plt.title(start_date.strftime("%A %B %d, %Y"))

In [ ]:
# The length of the text for each widget.
style = {'description_width': '150px'}

# Area selector (Streetscape, Under Raincoat, Outside).
scenes_widget = widgets.Dropdown(options=SCENES, 
                                 tooltip="Areas captured by sensors at 307",
                                 style=style, 
                                 description='Area')

# Tracked object type selector (pedestrian, bicycle, car, bus, truck).
object_class_widget = widgets.Dropdown(options=OBJECT_CLASSES_CAPITALIZED, 
                                       style=style, 
                                       description='Object Type')

# Zone metric selector (object count, mean dwell time, absorption index).
metrics_widget = widgets.Dropdown(options=EXTENDED_METRICS, 
                                  style=style, 
                                  description='Metric')

# Manual date input that is activated if date slider mode is not checked.
date_picker_widget = widgets.DatePicker(value=START_DATE,
    description='Date',
    disabled=False,
    continuous_update=False
)

# Controls whether to hide or show events.
show_events_widget = widgets.Checkbox(value=False, 
                                      description="Show Events Only")

# Controls whether to hide or show more options.
advanced_options_widget = widgets.Checkbox(value=False, 
                                           description="Show Advanced Options")

# Selects one of the weekdays or show all days.
weekdays_widget = widgets.Dropdown(options=WEEKDAYS_DROPDOWN, 
                                   rows=7, 
                                   description='Weekday', 
                                   style=style, 
                                   disabled=False)

# Can be either relative or absolute
display_mode_widget = widgets.Dropdown(options=DISPLAY_MODES, 
                                       style=style, 
                                       description='Display Mode')

# If absolute mode is selected, then this controls the constants all zone values are compared against.
max_zone_val_widget = widgets.BoundedIntText(min=0, 
                                            max=2000, 
                                            value=200, 
                                            step=1,
                                            description="Maximum Zone Value", 
                                            style=style, 
                                            disabled=True)

# A value set here is the minimum objects that need to pass through a zone for the zone to be displayed.
min_object_widget = widgets.BoundedIntText(min=0, 
                                           value=0, 
                                           step=1, 
                                           style=style, description="Minimum Zone Objects")

# The number of zones that are shown.
zones_shown_widget = widgets.BoundedIntText(min=1, 
                                            max=ZONE_NUM, 
                                            value=ZONE_NUM, 
                                            step=1, 
                                            style=style, 
                                            description="Zones Shown")

# The method by which the subset of zones is selected if Zones Shown is used. The methods are highest, 
# lowest, and middle values.
zone_quality_widget = widgets.Dropdown(options=ZONE_QUALITIES, 
                                       style=style, 
                                       description='Zone Quality')

# Controls whether date selection is done using the slider or by manual input.
time_mode_widget = widgets.Checkbox(value=True, 
                                    description="Use Date Slider")

# A slider that controls which day's data to display in the heatmap.
date_slider = widgets.IntSlider(min=-1, 
                                max=FRAMES-1, 
                                value=-1, 
                                step=1, 
                                description="", 
                                style=style, 
                                readout=False)

# Provides controls for the slider including a stop, start, reset, and looping buttons.
date_animate = widgets.Play(value=-1, 
                            min=-1, 
                            max=FRAMES-1, 
                            step=1, 
                            interval=250, style=style, 
                            description="Play")

# Synchronizes the play buttons with the slider value.
widgets.jslink((date_animate, 'value'), (date_slider, 'value'))
widgets.jslink((date_animate, 'max'), (date_slider, 'max'))

# The play buttons and slider are disabled or not disabled together based on the date slider mode checkbox.
widgets.jslink((date_animate, 'disabled'), (date_slider, 'disabled'))

# Widgets that are hidden or shown based on the Show Advanded Widgets checkbox.
advanced_widgets = [display_mode_widget, max_zone_val_widget, min_object_widget, 
                    zone_quality_widget, zones_shown_widget]


# The maximum zone widget is used only when the display mode is absolute and is disabled otherwise.
def change_lock_state(*args):
    max_zone_val_widget.disabled = (display_mode_widget.value == 'Relative')

# Change the disabled status of max_zone_val_widget every time the display mode is changed.
display_mode_widget.observe(change_lock_state, 'value')

# When a weekday is selected, only frames corresponding to that weekday are shown. 
def change_animation_variables(*args):
    # The default is all frames are shown.
    if weekdays_widget.value == "All":
        date_animate.step = 1
        date_animate.min = -1
    # A weekday is selected.
    else:
        # This check ensures the next frame at the time of selecting a new weekday is the first frame after the 
        # current frame that is on the selected weekday. The cases are if the current weekday index is ahead 
        # or behind the selected weekday index.
        if date_animate.value % 7 > WEEKDAYS_TO_NUM[weekdays_widget.value]:
            date_animate.value = (int(date_animate.value/7) + 1) * 7 + WEEKDAYS_TO_NUM[weekdays_widget.value]
        else:
            date_animate.value = int(date_animate.value/7) * 7 + WEEKDAYS_TO_NUM[weekdays_widget.value]
        # These settings ensure only the selected weekday is shown during the first playthrough and after the reset.
        date_animate.step = 7
        date_animate.min = WEEKDAYS_TO_NUM[weekdays_widget.value]
    
    
# Change the animation position and increment whenever a new weekday or "All" is selected in the weekdays selector.
weekdays_widget.observe(change_animation_variables, 'value')

# Update the date slider parameters based on the value of the Show Events Only checkbox.
def change_event_parameters(*args):
    # If only events are shown, then the heatmap will go through the events in the event list and there are
    # NUM_EVENTS number of them. The value of the slider is the index in the events list. Since a blank heatmap will
    # be shown at the beginning, set the minimum slider to 0 rather than -1.
    if show_events_widget.value:
        date_animate.max = NUM_EVENTS - 1
        date_animate.min = 0
    # In this case, both events and non-events are shown so the slider will move through FRAMES days with a blank
    # heatmap at the start.
    else:
        date_animate.max = FRAMES - 1
        date_animate.min = -1
    # Whenever the Show Events Only checkbox changes value, the slider is reset to the start.
    date_animate.value = date_animate.min
        
# Change the animation position and days shown whenever the Show Events Only checkbox is checked or unchecked.
show_events_widget.observe(change_event_parameters, 'value')

# Use the widgets to control the zone heatmap.
interactive_visual = widgets.interactive_output(show_dwell_stats, {
    'scene': scenes_widget,
    'object_class': object_class_widget,
    'metric': metrics_widget,
    'chosen_date': date_picker_widget,
    'show_events': show_events_widget,
    'date_slider_mode': time_mode_widget,
    'advanced_options': advanced_options_widget,
    'display_mode': display_mode_widget,
    'max_abs_val': max_zone_val_widget,
    'weekdays': weekdays_widget,
    'min_objects': min_object_widget,
    'zone_quality': zone_quality_widget,
    'zones_shown': zones_shown_widget,
    'frame': date_slider})

# The positioning the widgets in on screen. The are placed in two columns, with the main controls in the left column
# and advanced widgets in the right column.
interactive_visual_ui = widgets.VBox([widgets.HBox(
    [widgets.VBox([scenes_widget, object_class_widget, metrics_widget, weekdays_widget, show_events_widget, 
                   time_mode_widget, date_picker_widget]),
    widgets.VBox([advanced_options_widget, min_object_widget, zones_shown_widget, zone_quality_widget, 
                  display_mode_widget, max_zone_val_widget],
                layout=widgets.Layout(left="1cm"))]), 
                                      widgets.HBox([date_animate, date_slider])])

In [ ]:
# Shows the traffic or dwell time data for each day in a heatmap with rows representing weekdays.
def show_day_heatmap(scene, object_class, metric, show_events):
    # The total number of smaller rectangles in the heatmap. Since the heatmap is rectangular, there may be more 
    # smaller rectangles in the heatmap than days.
    heatmap_days = math.ceil(FRAMES/7) * 7
    # The number of smaller rectangles across each row in the heatmap.
    days_per_weekday = math.ceil(FRAMES/7)
    # Get the days whose data are tracked to be used as the x-axis of the heatmap.
    dates = START_DATE + np.arange(days_per_weekday) * timedelta(7)
    # These arrays store the heatmap values for all heatma_days days.
    weekday_classes = np.zeros((7, days_per_weekday))
    
    # Convert the upper case selected object class to a lower case version used for indexing.
    converted_object_class = OBJECT_CLASSES_CONVERTER[object_class]
    
    # Set the heatmap value for each of the smaller rectangles.
    for day_index in range(heatmap_days):
        # Get the date corresponding to the current day index. The date corresponding to the lower-left corner 
        # is the first date, which is February 18th, 2019. 
        day_date = date(2019, 2, 18) + timedelta(day_index)
        # If the date corresponding to the rectangle falls outside the tracked date range, set the date's value to 0.
        if day_date >= START_DATE and day_date <= END_DATE:
            # Get the data value corresponding to this day.
            day_data = AREA_DATA[converted_object_class][METRICS.index(
            metric)][SCENE_TO_NUM[scene]][(day_date - START_DATE).days]
            # Set the data in the array location corresponding to the day.
            weekday_classes[day_index % 7][int(day_index/7)] = day_data
            # If Show Events is checked and the day is not an event, set the value in the heatmap to 0.
            if show_events and day_date not in EVENT_DATES:
                weekday_classes[day_index % 7][int(day_index/7)] = 0
            
    # The text on a rectangle uponing over the mouse of it will display the date the rectangle corresponds to and the
    # value for that date.
    hovertexts = []
    
    # For each weekday, find the list of days in the heatmap corresponding to that weekday.
    for base_index in range(7):
        weekday_texts = []
        # The first day in the heatmap that falls on the weekday.
        base_date = date(2019, 2, 18) + timedelta(base_index)
        for weekday_date_index in range(days_per_weekday):
            # Consecutive days in the same weekday are separated by seven days.
            weekday_date_text = base_date + weekday_date_index * timedelta(7)
            # Add the text for that date into the text array for this weekday.
            weekday_texts.append("<br>Date: " + weekday_date_text.strftime("%B %d, %Y") + "<br>Value: " 
                                 + str(weekday_classes[base_index][weekday_date_index]))
        # Add all the exts for the weekday into the list of texts for all days.
        hovertexts.append(weekday_texts)
          
    # Create a base figure to add to.
    fig = go.Figure()
    
    # Add a heatmap showing the values for every day with low values represented by white and high values represented
    # by blue.
    fig.add_trace(go.Heatmap(
            z=weekday_classes,
            x=dates,
            y=WEEKDAYS,
            colorscale=[[0, 'rgba(255, 255, 255, 1)'], 
               [1, 'rgba(0, 0, 255, 1)']],
            hoverinfo='text',
            text=hovertexts))

    # Center the title and show the months on the x-axis.
    fig.update_layout(
        title=metric + " for " + object_class + " in " + scene + " Each Day", 
        title_x=0.5,
        xaxis_nticks=12)

    fig.show()

In [ ]:
# Area selector (Streetscape, Under Raincoat, Outside).
scenes_widget = widgets.Dropdown(options=SCENES, 
                                 tooltip="Areas captured by sensors at 307", 
                                 style=style, 
                                 description='Area' )


# Tracked object type selector (pedestrian, bicycle, car, bus, truck).
object_class_widget = widgets.Dropdown(options=OBJECT_CLASSES_CAPITALIZED, 
                                       style=style, 
                                       description='Object Type')

# Zone metric selector (object count, mean dwell time, absorption index).
metrics_widget = widgets.Dropdown(options=METRICS, 
                                  style=style, 
                                  description='Metric')

# Controls whether to hide or show events.
show_events_widget = widgets.Checkbox(value=False, 
                                      description="Show Events Only")

In [ ]:
zone_heatmap_out = widgets.Output(layout={'margin': '1px solid black'})
day_heatmap_out = widgets.Output(layout={'margin': '1px solid black'})

with zone_heatmap_out:
    display(interactive_visual, interactive_visual_ui)
with day_heatmap_out:
    # Use the widgets to control the day heatmap.
    _ = widgets.interact(show_day_heatmap, scene=scenes_widget, object_class=object_class_widget, 
                                      metric=metrics_widget, show_events=show_events_widget)